<font face="Calibri" size="2"> <i>eSBAE - Notebook Series - Part 3, version 0.4, March 2023. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![titre](images/header.png)

# III - Extraction de séries temporelles eSBAE
### Extraire diverses données de séries chronologiques pour de grands ensembles de points à partir de Google Earth Engine
-------

Ce bloc-notes vous guide tout au long du processus d'extraction de séries chronologiques pour un ensemble de points à l'aide de [Google's Earth Engine](https://earthengine.google.com/). Le script est optimisé pour traiter des milliers de points et utilisera la parallélisation pour extraire efficacement les informations de la plateforme.

**Tu auras besoin de**:
- un compte Earth Engine valide ([inscrivez-vous ici](https://code.earthengine.google.com/register))
- un tableau de points téléchargé (Feature Collection)
- la table a besoin d'un identifiant de point unique (Point ID)

**Vous devez savoir que :**

- En tant qu'utilisateur SEPAL : ce notebook ne nécessite **pas d'énormes ressources**, car les traitements sont effectués sur la plate-forme. Une instance **m2** est la mieux adaptée.
- L'extraction peut prendre jusqu'à plusieurs jours (>100000 points). Si vous êtes sur SEPAL, utilisez l'option **"maintenir l'exécution de l'instance"** dans le tableau de bord du rapport utilisateur. Cependant, **n'oubliez pas** d'éteindre votre machine une fois le traitement terminé.
- Un fichier journal est créé dans votre dossier tmp. L'interruption de la connectivité au serveur SEPAL peut conduire à bloquer la sortie du notebook Jupyter. **Cela ne signifie pas que le traitement s'est arrêté.** Vous pouvez voir dans esbae_log_(time) si le traitement est toujours en cours.
- Vous pouvez redémarrer le noyau et exécuter toutes les cellules, et l'extraction **commencera là où elle s'est arrêtée**. Ceci est également valable si votre instance a été arrêtée avant que le traitement ne soit complètement terminé.

### 1 - Importer des bibliothèques (*exécuter uniquement cette cellule*)

Cette cellule nous fournira les fonctionnalités dont nous avons besoin pour exécuter les cellules suivantes du notebook.

In [1]:
# initialize EE    
import ee
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
    
from sampling_handler import TimeSeriesExtraction

/usr/local/share/jupyter/kernels/venv-eSBAE_notebooks/venv/lib/python3.8/site-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Variables d'entrée de base

Ici, une soi-disant instance de classe est initialisée. L'instance de classe nécessite la définition de certains paramètres et est écrite dans la variable *esbae*.

In [2]:
esbae = TimeSeriesExtraction(
     # your project name that you use fo all of the notebooks
    project_name  = 'esbae_congo',
    
    # your start and end date. 
    # NOTE that this should go further back to the past than the 
    # envisaged monitoing period for calibration purposes
    ts_start      = '2015-01-01',      # YYYY-MM-DD format
    ts_end        = '2023-01-01',        # YYYY-MM-DD format
    
    # satellite platform (for now only Landsat is supported)
    satellite     = 'Landsat',
    
    # at what resolution in metres you want to extract (shall conform with forest definition MMU)
    scale         = 70, # pixel size in metres
    
    # wether the TS will be extracted on a bounding box with diameter scale with original scale (e.g 30m for Landsat) of the underlying data (True), 
    # or if the underlying data is rescaled to the scale (False)
    # setting it to True might be more accurate, but tends to be slower
    bounds_reduce = False,
    
    # bands
    bands         =  [
        'green', 'red', 'nir', 'swir1', 'swir2',   # reflectance bands
        'ndfi', #'ndmi', 'ndvi',                    # indices
        #'brightness', 'greenness', 'wetness'       # Tasseled Cap 
    ], 
    # This is in case you haven't run notebook 1 and 2, and want to directly start from here
    #aoi = ee.FeatureCollection(ee.FeatureCollection('users/username/my_points').geometry().convexHull(100))
    
)

INFO: Using existing project directory at /home/sepal-user/module_results/esbae/esbae_congo
INFO: Using existent config file from project directory /home/sepal-user/module_results/esbae/esbae_congo


### 3 - Paramètres Landsat

Ici, vous pouvez sélectionner les satellites que vous souhaitez inclure dans la mission Landsat.
De plus, vous pouvez sélectionner la correction BRDF et un filtre pour une couverture nuageuse maximale. Notez que le paramètre bands est déjà défini lors de l'initialisation et sera extrait de l'attribut class.

In [3]:
# landsat related parameters
lsat_params = {
    'l9': True,
    'l8': True,
    'l7': True,
    'l5': True,
    'l4': True,
    'brdf': True,
    'bands': esbae.bands,
    'max_cc': 75    # percent
} 

# apply the basic configuration set in the cell above
esbae.lsat_params = lsat_params

### 4 - Paramètres de traitement

Ici, vous pouvez affiner les options de parallélisation. Pour une extraction efficace, l'extraction de séries chronologiques est effectuée sur des blocs de données, définis par des grilles carrées de tailles données. La routine vérifiera le nombre de points dans chaque bloc. Si cela est inférieur au max_points_per_chunk, il continuera et traitera ces points. Sinon, il essaiera de traiter ces points à un niveau de taille de grille inférieur. Certains paramètres optimisés sont donnés ci-dessous, commentez et décommentez le cas échéant.

In [4]:
esbae.workers = 10                   # this defines how many parallel requests will be send to EarthEngine at a time
esbae.max_points_per_chunk = 100     # this defines the maximum amount of points as send per request to Earth Engine at a time

# this defines the chunk sizes (in degree) to create the requests
# esbae.grid_size_levels = [0.1, 0.075, 0.05]   # optimized for 1km systematic grid
# esbae.grid_size_levels = [0.2, 0.15, 0.1]    # optimized for 2km systematic grid
esbae.grid_size_levels = [0.4, 0.3, 0.2]     # optimized for 4km systematic grid

### 5 - Définir une grille personnalisée

Cette étape n'est nécessaire que si vous avez ignoré le bloc-notes 2. Vous devez ensuite définir une collection d'entités Earth Engine ainsi que l'identifiant de point unique. Décommentez les lignes en supprimant le #

In [8]:
#esbae.sample_asset = 'users/raymond/esbae_congo/centroid_samples_ISEA3H_12'
#esbae.pid = 'point_id'

### 4 - Vérifier les données déjà traitées (optionnel)

Ceci est utile pour les grandes tailles de points et lorsque la connexion à Sepal est interrompue. Habituellement, le traitement se poursuit, mais il n'est pas simple de suivre les progrès.
Vous pouvez à la place redémarrer le noyau, exécuter toutes les cellules et voir si le traitement est terminé avec la ligne de code suivante.

In [10]:
esbae.check_if_completed()

INFO: Verifying 
INFO: No time-series data has been extracted yet.


### 5 - Exécutez l'extraction de données de séries chronologiques * (exécutez uniquement ceci) *

In [11]:
esbae.get_time_series_data()

INFO: Verifying parameter settings...
INFO: Preparing the processing of 3570 points.
INFO: No time-series data has been extracted yet.
INFO: Create AOI from points and upload as temporary EE asset inside tmp_esbae_230525_115106.
INFO: Creating processing chunks of 0.4 degrees for parallel extraction.
INFO: Preparing the parallel extraction over a total of 218 chunks. This may take a while...
INFO: Starting the parallel extraction routine.
INFO: Extracting 18 points for chunk nr 8 at resolution 0.4
INFO: Extracting 5 points for chunk nr 0 at resolution 0.4
INFO: Extracting 19 points for chunk nr 4 at resolution 0.4
INFO: Extracting 2 points for chunk nr 2 at resolution 0.4
INFO: Extracting 6 points for chunk nr 6 at resolution 0.4
INFO: Extracting 9 points for chunk nr 7 at resolution 0.4
INFO: Extracting 23 points for chunk nr 9 at resolution 0.4
INFO: Extracting 7 points for chunk nr 1 at resolution 0.4
INFO: Extracting 20 points for chunk nr 3 at resolution 0.4
INFO: Extracting 2 poi